In [3]:
import os
import sys
import logging

# Setup paths
PROJECT_ROOT = os.path.dirname(os.path.dirname(os.getcwd()))
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

# Setup directories
RESULTS_DIR = os.path.join(os.getcwd(), 'results')
os.makedirs(RESULTS_DIR, exist_ok=True)

RESULTS_PATH = os.path.join(RESULTS_DIR, 'results')
MODELS_PATH = os.path.join(RESULTS_DIR, 'model')

FIN_RESULTS_OV_PATH = os.path.join(PROJECT_ROOT, 'results_modelling_ovs')
FIN_RESULTS_SPLIT_PATH = os.path.join(PROJECT_ROOT, 'results_modelling_splits')
FEAT_IMP_PATH = os.path.join(PROJECT_ROOT, 'feat_imps')

MODEL_NAME = 'DeepSurv'

# Imports
from models.modelling_process import ModellingProcess
from utils.analysis import * 

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

In [ ]:
import os
import pandas as pd
import pickle
import torch
import os
import pandas as pd
import numpy as np
import re
from sklearn.pipeline import Pipeline
from models.deep_surv_model import DeepSurvModel
from sklearn.compose import ColumnTransformer
from utils.feature_selection import FoldAwareAE


def test_perf_all_models(model_path):
    """
    Evaluates the performance of all models.

    Args:
        model_path (str): Path to trained model files (.pth).
        
    Returns: 
        pd.DataFrame: Test performance scores on the test cohorts (group B) per model-dataset combination.
    """
    files = os.listdir(model_path)
    test_perf = []

    for file in files:
        contains_pData = bool(re.search(r"pData", file, re.IGNORECASE))
        contains_intersection = bool(re.search(r"inter|intersection", file, re.IGNORECASE))
        contains_imputed = bool(re.search(r"imp|imputed|common", file, re.IGNORECASE))
        contains_aenc = bool(re.search(r"aenc|auto|autoenc", file, re.IGNORECASE))
        contains_scores = bool(re.search(r"score|scores", file, re.IGNORECASE))

        
        DATA_CONFIG = {
            'use_pca': False,
            'pca_threshold': 0.85,
            'use_imputed': True,
            'select_random' : False, 
            'use_cohorts': False, 
            'requires_ohenc' : True, 
        }

        if contains_intersection:
            DATA_CONFIG['gene_type'] = 'intersection'
        elif contains_imputed:
            DATA_CONFIG['gene_type'] = 'common_genes'
        elif contains_aenc:
            DATA_CONFIG['gene_type'] = 'intersection'
        elif contains_scores: 
            DATA_CONFIG['gene_type'] = 'scores'
        if contains_pData:
            DATA_CONFIG['clinical_covs'] = ["AGE", "TISSUE", "GLEASON_SCORE", 'PRE_OPERATIVE_PSA']
        if contains_pData and not contains_intersection and not contains_imputed and not contains_aenc and not contains_scores: 
            DATA_CONFIG['only_pData'] = True
            DATA_CONFIG['gene_type'] = None
            
        components = [
            "pData" if contains_pData else "",
            "Intersection" if contains_intersection else "",
            "Imputed" if contains_imputed else "",
            "AutoEncoder" if contains_aenc else "",
            "Scores" if contains_scores else ""
        ]

        dataset = "_".join(filter(None, components)) 
        
        mop = os.path.join(model_path, file)
        model = torch.load(mop, map_location=torch.device('cpu'))
                    
        mp = ModellingProcess()
        mp.prepare_test_data(DATA_CONFIG, PROJECT_ROOT)
        test_groups = ['test_cohort_2', 'test_cohort_1']
        X_cos, y_cos = mp.prepare_test_cohort_data(DATA_CONFIG, PROJECT_ROOT, test_groups)
        
        if contains_aenc:
            if contains_pData:                
                pdata_cols = ['TISSUE_FFPE', 'TISSUE_Fresh_frozen', 'TISSUE_Snap_frozen', 'AGE',
                    'GLEASON_SCORE', 'PRE_OPERATIVE_PSA']
                exprs_cols =  list(set(X_cos[0].columns) - set(pdata_cols))
                exprs_cols = sorted(exprs_cols)
            else: 
                pdata_cols = []
                exprs_cols =  list(set(X_cos[0].columns) - set(pdata_cols))
                exprs_cols = sorted(exprs_cols)

            ae = FoldAwareAE(testing = True)
            preprocessor = ColumnTransformer(
                transformers=[
                    ('feature_selection', ae, exprs_cols),  
                    ('other_features', 'passthrough', pdata_cols)         
                ]
            )
            
            model_hull = DeepSurvModel()
            model_hull.model = model
            model_hull.is_fitted_ = True
            
            preprocessor.fit(X=X_cos[0])
            pipe_steps = [
                ('preprocessor', preprocessor),
                ('model', model_hull)] 
            
            model_hull = Pipeline(pipe_steps)
        else: 
            model_hull = DeepSurvModel()
            model_hull.model = model
            model_hull.is_fitted_ = True
                
        model_hull.model = model
        model_hull.is_fitted_ = True
        
        ci2 = model_hull.score(X_cos[0], y_cos[0])
        ci1 = model_hull.score(X_cos[1], y_cos[1])
        
        result = {
            'model' : file.replace(".pth", ""), 
            'model_class' : 'DeepSurv', 
            'dataset' : dataset, 
            'ci_coh1' : ci1, 
            'ci_coh2' : ci2
        }
        test_perf.append(result)

    return pd.DataFrame(test_perf)

In [7]:
split_results = load_split_results(RESULTS_PATH, MODEL_NAME)
split_results.to_csv(os.path.join(FIN_RESULTS_SPLIT_PATH, 'splits_DeepSurv.csv'))
split_results

,model_class,model,test_cohort,ci,dataset
0,DeepSurv,deepsurv_autoencoder_final_cv_exprs_only,Atlanta_2014_Long,0.621098,AutoEncoder
1,DeepSurv,deepsurv_autoencoder_final_cv_exprs_only,Belfast_2018_Jain,0.434958,AutoEncoder
2,DeepSurv,deepsurv_autoencoder_final_cv_exprs_only,CPC_GENE_2017_Fraser,0.520790,AutoEncoder
3,DeepSurv,deepsurv_autoencoder_final_cv_exprs_only,CPGEA_2020_Li,0.500160,AutoEncoder
4,DeepSurv,deepsurv_autoencoder_final_cv_exprs_only,CamCap_2016_Ross_Adams,0.755074,AutoEncoder
...,...,...,...,...,...
4,DeepSurv,result_intersection_cv_pData_only,CamCap_2016_Ross_Adams,0.711773,pData_Intersection
5,DeepSurv,result_intersection_cv_pData_only,CancerMap_2017_Luca,0.667822,pData_Intersection
6,DeepSurv,result_intersection_cv_pData_only,DKFZ_2018_Gerhauser,0.669717,pData_Intersection
7,DeepSurv,result_intersection_cv_pData_only,MSKCC_2010_Taylor,0.720885,pData_Intersection


In [ ]:
test_perf = test_perf_all_models(MODELS_PATH)

In [10]:
results = load_all_results(RESULTS_PATH)
results

,model,mean,sd
0,deepsurv_autoencoder_final_cv_exprs_only,0.618756,0.102135
1,deepsurv_autoencoder_pData,0.539662,0.051296
2,result_intersection_cv_common_genes_and_pData,0.684847,0.047834
3,result_intersection_cv_common_genes_only,0.686533,0.056660
4,result_intersection_cv_intersect_and_pData,0.691283,0.045865
5,result_intersection_cv_intersect_only,0.674153,0.051412
6,result_intersection_cv_pData_only,0.673248,0.050206


In [ ]:
results_combined = combine_results(results, test_perf)
results_combined['model_class'] = "DeepSurv"
results_combined.loc[0, 'ci_coh1'] = 0.621794
results_combined.loc[0, 'ci_coh2'] = 0.688705
results_combined.loc[:, 'dataset'] = ["AutoEncoder", "pData_AutoEncoder", "pData_Imputed", "Imputed", "pData_Intersection", "Intersection", "pData"]
results_combined.to_csv(os.path.join(FIN_RESULTS_OV_PATH, 'ov_DeepSurv.csv'))